In [ ]:
!pip install bert-tensorflow
!pip install transformers

     |████████████████████████████████| 71kB 5.3MB/s 


In [ ]:
# Importing the libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import string
import nltk 
from nltk.corpus import stopwords
from tqdm import tqdm
from nltk.stem import PorterStemmer
import re
import os
import sys
import json
import tensorflow_hub as hub
import tensorflow as tf
import bert
from bert import tokenization
FullTokenizer = bert.tokenization.FullTokenizer
import argparse
from transformers import BertTokenizer
from tensorflow.keras.models import Model
from bert import optimization
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir('/content/drive/My Drive')

In [ ]:
data = pd.read_csv("open/train.csv")
test = pd.read_csv("open/test_x.csv")

print("Size of the data : ", data.shape)
print("Size of the Test data : ", test.shape)
# there are 54879 texts

Size of the data :  (54879, 3)
Size of the Test data :  (19617, 2)


In [ ]:
data.head(10)

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3
5,5,"""It was well fought,"" he said, ""and, by my soo...",4
6,6,"Not to pay him was impossible, considering his...",3
7,7,"“A proper figure of a man at-arms,” said the l...",2
8,8,"'You were not here last Sunday night,' he said.",0
9,9,“You must not ask me that!” I cried. “Hell may...,4


In [ ]:
max_seq_length = 212
input_word_ids = tf.keras.layers.Input(shape =(max_seq_length,), dtype = tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape =(max_seq_length,), dtype=tf.int32,
                                   name = "input_mask")
segment_ids = tf.keras.layers.Input(shape =(max_seq_length,),dtype =tf.int32,
                                  name = "segment_ids")
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable = True)
pooled_output, sequence_output = bert_layer([input_word_ids,input_mask,segment_ids])

model = Model(inputs=[input_word_ids,input_mask,segment_ids],outputs=[pooled_output,sequence_output])


In [ ]:
def get_masks(tokens, max_seq_length):
  """Mask for padding"""
  if len(tokens) > max_seq_length:
    raise IndexError("Token length more than max seq length!")
  return [1]*len(tokens) + [0]*(max_seq_length-len(tokens))

def get_segments(tokens, max_seq_length):
  """segments:0 for the first sequence, 1 for the second"""
  if len(tokens) > max_seq_length:
    raise IndexError("Token length more than max seq length!")
  segments = []
  current_segment_id = 0
  for token in tokens:
    segments.append(current_segment_id)
    if token == "[SEP]":
      current_segment_id = 1
  return segments+ [0] * (max_seq_length - len(tokens))


def get_ids(tokens, tokenizer, max_seq_length):
  """Token ids form Tokenizer vocab"""
  token_ids = tokenizer.convert_tokens_to_ids(tokens)
  input_ids = token_ids + [0] * (max_seq_length - len(token_ids))
  return input_ids
  

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
s  = data['text'][0]
stokens = tokenizer.tokenize(s)
stokens = ["[CLS]"] + stokens + ["[SEP]"]

input_ids =  np.asarray(get_ids(stokens, tokenizer, max_seq_length))
input_masks = np.asarray(get_masks(stokens, max_seq_length))
input_segments = np.asarray(get_segments(stokens, max_seq_length))

pool_embs, all_embs = model.predict([input_ids,input_masks,input_segments])

In [ ]:
len(all_embs)

212

In [ ]:
tz = BertTokenizer.from_pretrained("bert-base-cased")
stokens = tz.tokenize(s)

In [ ]:
stokens